In [92]:
import requests
import json
import pandas as pd

def get_imf_indicators(type_):
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    indicators_df=indicators_df[indicators_df["source"].str.contains(type_, na=False)]
    return indicators_df


def get_imf_groups(group_type):
    response = requests.get(
        url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}"
    )
    response_body = json.loads(response.text)
    results = [
        {"id": key, **values} for key, values in response_body[group_type].items()
    ]
    df = pd.DataFrame.from_records(results)
    return df

indicators = get_imf_indicators("World Economic Outlook")
indicators
countries = get_imf_groups('countries')
countries

weo=indicators[indicators["source"].str.contains("World Economic Outlook", na=False)]
weo

weo_gdp=weo[weo["label"]=="Real GDP growth"]
weo_gdp



def get_imf_indicator_data(
    indicator_id: str, group_ids: list[str] = [], years: list[str] = []
):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = "?periods=" + ",".join(years)

    response = requests.get(
        url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}"
    )
    response_body = json.loads(response.text)
    response_values = response_body.get("values")

    if not response_values:
        return pd.DataFrame()

    indicator_df = pd.DataFrame.from_records(
        response_body["values"][indicator_id]
    ).sort_index()

    return indicator_df


indicators = get_imf_indicators()
countries = get_imf_groups('countries')

indicator = indicators.loc[indicators.id == 'PCPIEPCH'].iloc[0]

selected_countries = countries.loc[countries.id.isin(['TUR','FRA',"DEU","USA","GBR","GR","LBY","UKR",
                                                      "ITA","IRQ","ESP","NLD","RUS","ROU","IRN",
                                                      "ARE","BEL","POL","EGY","BGR","CHN","ISR"])]

# 3rd step: fetch indicator data
indicator_data = get_imf_indicator_data(indicator.id, selected_countries.id)

indicator_data.loc['2023':].T.to_excel("big_partners_.xlsx")

id                                                      PCPIEPCH
label              Inflation rate, end of period consumer prices
description    The end of period consumer price index (CPI) i...
source                       World Economic Outlook (April 2025)
unit                                       Annual percent change
dataset                                                      WEO
Name: 8, dtype: object


,ABW,ADVEC,AFG,AFQ,AGO,ALB,AND,APQ,ARE,ARG,...,WBG,WE,WEOWORLD,WEQ,WHQ,WSM,YEM,ZAF,ZMB,ZWE
1980,NaN,1.3,NaN,NaN,2.4,2.7,NaN,4.6,-1.8,0.7,...,NaN,6.3,2.2,1.7,2.1,-6.1,NaN,6.6,3.9,NaN
1981,NaN,2.3,NaN,NaN,-4.4,5.7,NaN,5.2,8.0,-5.7,...,NaN,1.4,2.3,1.1,2.2,-9.1,NaN,5.4,6.6,NaN
1982,NaN,0.2,NaN,NaN,0.0,2.9,NaN,4.2,-7.2,-3.1,...,NaN,-0.4,0.7,0.8,-1.4,-1.0,NaN,-0.4,-2.9,NaN
1983,NaN,3.2,NaN,NaN,4.2,1.1,NaN,5.5,-5.2,3.7,...,NaN,-3.0,2.7,1.8,2.0,0.5,NaN,-1.8,-1.1,NaN
1984,NaN,4.8,NaN,NaN,6.0,2.0,NaN,6.4,4.5,2.0,...,NaN,3.9,4.6,2.5,6.1,2.0,NaN,5.1,-1.7,NaN
1985,NaN,3.7,NaN,NaN,3.5,-1.5,NaN,5.8,-2.5,-7.0,...,NaN,2.8,3.7,2.7,3.8,5.8,NaN,-1.2,1.2,NaN
1986,NaN,3.3,NaN,NaN,2.9,5.6,NaN,5.4,-19.3,7.1,...,NaN,3.8,3.4,2.7,3.5,4.8,NaN,0.0,1.7,NaN
1987,16.1,3.8,NaN,NaN,4.1,-0.8,NaN,6.8,5.3,2.5,...,NaN,3.3,3.9,3.0,3.5,0.5,NaN,2.1,1.5,NaN
1988,18.6,4.7,NaN,NaN,6.1,-1.4,NaN,7.7,-2.6,-2.0,...,NaN,1.0,4.5,4.2,3.2,2.9,NaN,4.2,9.3,NaN
1989,12.1,4.0,NaN,NaN,0.0,9.8,NaN,5.4,15.7,-7.0,...,NaN,0.9,3.7,3.8,2.8,5.8,NaN,2.4,-3.7,NaN


In [90]:
indicators

,id,label,description,source,unit,dataset
0,NGDP_RPCH,Real GDP growth,Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Annual percent change,WEO
1,NGDPD,"GDP, current prices",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Billions of U.S. dollars,WEO
2,NGDPDPC,"GDP per capita, current prices\n",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),U.S. dollars per capita,WEO
3,PPPGDP,"GDP, current prices",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Purchasing power parity; billions of internati...,WEO
4,PPPPC,"GDP per capita, current prices",Gross domestic product (GDP) is the most commo...,World Economic Outlook (April 2025),Purchasing power parity; international dollars...,WEO
5,PPPSH,"GDP based on PPP, share of world",Purchasing Power Parity (PPP) weights are indi...,World Economic Outlook (April 2025),Percent of World,WEO
6,PPPEX,Implied PPP conversion rate,Purchasing Power Parity (PPP) is a theory whic...,World Economic Outlook (April 2025),National currency per international dollar,WEO
7,PCPIPCH,"Inflation rate, average consumer prices",The average consumer price index (CPI) is a me...,World Economic Outlook (April 2025),Annual percent change,WEO
8,PCPIEPCH,"Inflation rate, end of period consumer prices",The end of period consumer price index (CPI) i...,World Economic Outlook (April 2025),Annual percent change,WEO
9,LP,Population,"Total population of a country, region, or grou...",World Economic Outlook (April 2025),Millions of people,WEO


In [15]:
weo.id.unique()


array(['NGDP_RPCH', 'NGDPD', 'NGDPDPC', 'PPPGDP', 'PPPPC', 'PPPSH',
       'PPPEX', 'PCPIPCH', 'PCPIEPCH', 'LP', 'BCA', 'BCA_NGDPD', 'LUR',
       'GGXCNL_NGDP', 'GGXWDG_NGDP'], dtype=object)

In [1]:
import requests
import json
import pandas as pd

def get_imf_indicators():
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    return indicators_df


def get_imf_groups(group_type: str):
    response = requests.get(
        url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}"
    )
    response_body = json.loads(response.text)
    results = [
        {"id": key, **values} for key, values in response_body[group_type].items()
    ]
    df = pd.DataFrame.from_records(results)
    return df



def get_imf_indicator_data(
    indicator_id: str, group_ids: list[str] = [], years: list[str] = []
):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = "?periods=" + ",".join(years)

    response = requests.get(
        url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}"
    )
    response_body = json.loads(response.text)
    response_values = response_body.get("values")

    if not response_values:
        return pd.DataFrame()

    indicator_df = pd.DataFrame.from_records(
        response_body["values"][indicator_id]
    ).sort_index()

    return indicator_df


indicators = get_imf_indicators()
indicators
countries = get_imf_groups('countries')
countries

weo=indicators[indicators["source"]== "World Economic Outlook (October 2024)"]
weo



countries = get_imf_groups('countries')
ids=indicators[indicators["source"]=="World Economic Outlook (October 2024)"].id.unique()

df=pd.DataFrame()
for i in ids:
    indicator = indicators.loc[indicators.id == i].iloc[0]
    selected_countries = countries.loc[countries.id.isin(['USA'])]

    indicator_data = get_imf_indicator_data(indicator.id, selected_countries.id)
    indicator_data.columns = [str(i)]
    df=pd.concat([indicator_data, df], axis=1)
    
df.sort_index(inplace=True)

In [2]:
df

,GGXWDG_NGDP,GGXCNL_NGDP,LUR,BCA_NGDPD,BCA,LP,PCPIEPCH,PCPIPCH,PPPEX,PPPSH,PPPPC,PPPGDP,NGDPDPC,NGDPD,NGDP_RPCH
1980,NaN,NaN,7.2,0.1,2.316,227.622,11.9,13.5,1,21.579,12552.943,2857.325,12552.943,2857.325,-0.3
1981,NaN,NaN,7.6,0.2,5.031,229.916,8.6,10.4,1,21.683,13948.701,3207.025,13948.701,3207.025,2.5
1982,NaN,NaN,9.7,-0.2,-5.533,232.128,4.0,6.2,1,21.185,14404.994,3343.800,14404.994,3343.800,-1.8
1983,NaN,NaN,9.6,-1.1,-38.695,234.246,3.9,3.2,1,21.607,15513.679,3634.025,15513.679,3634.025,4.6
1984,NaN,NaN,7.5,-2.3,-94.342,236.307,3.9,4.4,1,22.177,17086.441,4037.650,17086.441,4037.650,7.2
1985,NaN,NaN,7.2,-2.7,-118.159,238.416,3.3,3.5,1,22.300,18199.320,4339.000,18199.320,4339.000,4.2
1986,NaN,NaN,7.0,-3.2,-147.176,240.593,1.7,1.9,1,22.343,19034.766,4579.625,19034.766,4579.625,3.5
1987,NaN,NaN,6.2,-3.3,-160.661,242.751,4.2,3.6,1,22.252,20000.968,4855.250,20000.968,4855.250,3.5
1988,NaN,NaN,5.5,-2.3,-121.159,244.968,4.5,4.1,1,22.205,21375.999,5236.425,21375.999,5236.425,4.2
1989,NaN,NaN,5.3,-1.8,-99.485,247.286,4.9,4.8,1,22.220,22814.077,5641.600,22814.077,5641.600,3.7


In [1]:
#IMF Ekonomik Görünüm Türkiye Tahminleri
import requests
import json
import pandas as pd

def get_imf_indicators():
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    return indicators_df


def get_imf_groups(group_type: str):
    response = requests.get(
        url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}"
    )
    response_body = json.loads(response.text)
    results = [
        {"id": key, **values} for key, values in response_body[group_type].items()
    ]
    df = pd.DataFrame.from_records(results)
    return df



def get_imf_indicator_data(
    indicator_id: str, group_ids: list[str] = [], years: list[str] = []
):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = "?periods=" + ",".join(years)

    response = requests.get(
        url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}"
    )
    response_body = json.loads(response.text)
    response_values = response_body.get("values")

    if not response_values:
        return pd.DataFrame()

    indicator_df = pd.DataFrame.from_records(
        response_body["values"][indicator_id]
    ).sort_index()

    return indicator_df


indicators = get_imf_indicators()
indicators
countries = get_imf_groups('regions')
countries

weo=indicators[indicators["source"]== "World Economic Outlook (October 2024)"]
weo



countries = get_imf_groups('regions')
ids=indicators[indicators["source"]=="World Economic Outlook (October 2024)"].id.unique()

df = pd.DataFrame()

for i in ids:
    try:
        indicator = indicators.loc[indicators.id == i].iloc[0]
        selected_countries = countries.loc[countries.id.isin(['EUQ'])]
        indicator_data = get_imf_indicator_data(indicator.id, selected_countries.id)
        indicator_label = indicator["label"]  
        indicator_data.columns = [indicator_label]
        df = pd.concat([df, indicator_data], axis=1)
    except:
        pass

df.sort_index(inplace=True)
#df.to_excel("last_weo.xlsx")

In [16]:
import requests
import json
import pandas as pd
from tqdm import tqdm  # for progress bars

def get_imf_data(url: str) -> dict:
    """Generic function to fetch IMF API data"""
    response = requests.get(url)
    return json.loads(response.text)

def get_all_indicators() -> pd.DataFrame:
    """Get all IMF indicators with metadata"""
    data = get_imf_data("https://www.imf.org/external/datamapper/api/v1/indicators")
    return pd.DataFrame([{"id": k, **v} for k, v in data["indicators"].items()])

def check_country_has_data(indicator_id: str, country_code: str = "TUR", year: str = "2023") -> bool:
    """Check if a specific indicator has data for the given country"""
    try:
        url = f"https://www.imf.org/external/datamapper/api/v1/{indicator_id}/{country_code}?periods={year}"
        data = get_imf_data(url)
        return bool(data.get("values", {}).get(indicator_id, {}).get(country_code, {}).get(year))
    except:
        return False

def get_turkey_indicators(verbose: bool = True) -> pd.DataFrame:
    """Get all indicators that contain data for Turkey"""
    indicators = get_all_indicators()
    turkey_indicators = []
    
    # Use tqdm for progress bar if verbose
    iterable = tqdm(indicators.itertuples(), total=len(indicators)) if verbose else indicators.itertuples()
    
    for indicator in iterable:
        if check_country_has_data(indicator.id):
            turkey_indicators.append({
                "indicator_id": indicator.id,
                "indicator_label": indicator.label,
                "source": indicator.source,
                "unit": indicator.unit,
                "scale": indicator.scale
            })
    
    return pd.DataFrame(turkey_indicators)



# Usage:
turkey_indicators = get_turkey_indicators()
print(f"Found {len(turkey_indicators)} indicators with data for Turkey")

# Save to Excel
turkey_indicators.to_excel("turkey_indicators.xlsx", index=False)

# Optional: Filter by specific source (e.g. WEO)
weo_turkey_indicators = turkey

  0%|                                                                                          | 0/133 [00:02<?, ?it/s]


AttributeError: 'Pandas' object has no attribute 'scale'

In [17]:
import requests
import json
import pandas as pd
from tqdm import tqdm

def get_imf_data(url: str):
    """Generic function to fetch IMF API data"""
    response = requests.get(url)
    return json.loads(response.text)

def get_all_indicators():
    """Get all IMF indicators with metadata"""
    data = get_imf_data("https://www.imf.org/external/datamapper/api/v1/indicators")
    indicators = []
    for k, v in data["indicators"].items():
        
        indicator = {
            "id": k,
            "label": v.get("label", ""),
            "source": v.get("source", ""),
            "unit": v.get("unit", ""),
            "scale": v.get("scale", ""),
            "description": v.get("description", "")
        }
        indicators.append(indicator)
    return pd.DataFrame(indicators)

def check_country_has_data(indicator_id: str, country_code: str = "TUR", year: str = "2023"):
    """Check if a specific indicator has data for the given country"""
    try:
        url = f"https://www.imf.org/external/datamapper/api/v1/{indicator_id}/{country_code}?periods={year}"
        data = get_imf_data(url)
        return bool(data.get("values", {}).get(indicator_id, {}).get(country_code, {}).get(year))
    except:
        return False

def get_turkey_indicators(verbose: bool = True):
    """Get all indicators that contain data for Turkey"""
    indicators = get_all_indicators()
    turkey_indicators = []
    
    iterable = tqdm(indicators.itertuples(), total=len(indicators)) if verbose else indicators.itertuples()
    
    for indicator in iterable:
        if check_country_has_data(indicator.id):
            turkey_indicators.append({
                "indicator_id": indicator.id,
                "indicator_label": getattr(indicator, "label", ""),
                "source": getattr(indicator, "source", ""),
                "unit": getattr(indicator, "unit", ""),
                "scale": getattr(indicator, "scale", ""), 
                "description": getattr(indicator, "description", "")
            })
    
    return pd.DataFrame(turkey_indicators)


turkey_indicators = get_turkey_indicators()
print(f"Found {len(turkey_indicators)} indicators with data for Turkey")

turkey_indicators.to_excel("turkey_indicators.xlsx", index=False)

# Optional: Filter by specific source
if not turkey_indicators.empty:
    weo_turkey_indicators = turkey_indicators[turkey_indicators["source"].str.contains("World Economic Outlook", na=False)]
    print(f"Found {len(weo_turkey_indicators)} WEO indicators for Turkey")

100%|████████████████████████████████████████████████████████████████████████████████| 133/133 [05:54<00:00,  2.66s/it]

Found 49 indicators with data for Turkey
Found 15 WEO indicators for Turkey


In [18]:
turkey_indicators

,indicator_id,indicator_label,source,unit,scale,description
0,NGDP_RPCH,Real GDP growth,World Economic Outlook (April 2025),Annual percent change,,Gross domestic product is the most commonly us...
1,NGDPD,"GDP, current prices",World Economic Outlook (April 2025),Billions of U.S. dollars,,Gross domestic product is the most commonly us...
2,NGDPDPC,"GDP per capita, current prices\n",World Economic Outlook (April 2025),U.S. dollars per capita,,Gross domestic product is the most commonly us...
3,PPPGDP,"GDP, current prices",World Economic Outlook (April 2025),Purchasing power parity; billions of internati...,,Gross domestic product is the most commonly us...
4,PPPPC,"GDP per capita, current prices",World Economic Outlook (April 2025),Purchasing power parity; international dollars...,,Gross domestic product (GDP) is the most commo...
5,PPPSH,"GDP based on PPP, share of world",World Economic Outlook (April 2025),Percent of World,,Purchasing Power Parity (PPP) weights are indi...
6,PPPEX,Implied PPP conversion rate,World Economic Outlook (April 2025),National currency per international dollar,,Purchasing Power Parity (PPP) is a theory whic...
7,PCPIPCH,"Inflation rate, average consumer prices",World Economic Outlook (April 2025),Annual percent change,,The average consumer price index (CPI) is a me...
8,PCPIEPCH,"Inflation rate, end of period consumer prices",World Economic Outlook (April 2025),Annual percent change,,The end of period consumer price index (CPI) i...
9,LP,Population,World Economic Outlook (April 2025),Millions of people,,"Total population of a country, region, or grou..."


In [19]:
get_imf_data("https://www.imf.org/external/datamapper/api/v1/indicators")

{'indicators': {'NGDP_RPCH': {'label': 'Real GDP growth',
   'description': "Gross domestic product is the most commonly used single measure of a country's overall economic activity. It represents the total value at constant prices of final goods and services produced within a country during a specified time period, such as one year.",
   'source': 'World Economic Outlook (April 2025)',
   'unit': 'Annual percent change',
   'dataset': 'WEO'},
  'NGDPD': {'label': 'GDP, current prices',
   'description': "Gross domestic product is the most commonly used single measure of a country's overall economic activity. It represents the total value at current prices of final goods and services produced within a country during a specified time period, such as one year.",
   'source': 'World Economic Outlook (April 2025)',
   'unit': 'Billions of U.S. dollars',
   'dataset': 'WEO'},
  'NGDPDPC': {'label': 'GDP per capita, current prices\n',
   'description': "Gross domestic product is the most com

In [20]:
weo_turkey_indicators

,indicator_id,indicator_label,source,unit,scale,description
0,NGDP_RPCH,Real GDP growth,World Economic Outlook (April 2025),Annual percent change,,Gross domestic product is the most commonly us...
1,NGDPD,"GDP, current prices",World Economic Outlook (April 2025),Billions of U.S. dollars,,Gross domestic product is the most commonly us...
2,NGDPDPC,"GDP per capita, current prices\n",World Economic Outlook (April 2025),U.S. dollars per capita,,Gross domestic product is the most commonly us...
3,PPPGDP,"GDP, current prices",World Economic Outlook (April 2025),Purchasing power parity; billions of internati...,,Gross domestic product is the most commonly us...
4,PPPPC,"GDP per capita, current prices",World Economic Outlook (April 2025),Purchasing power parity; international dollars...,,Gross domestic product (GDP) is the most commo...
5,PPPSH,"GDP based on PPP, share of world",World Economic Outlook (April 2025),Percent of World,,Purchasing Power Parity (PPP) weights are indi...
6,PPPEX,Implied PPP conversion rate,World Economic Outlook (April 2025),National currency per international dollar,,Purchasing Power Parity (PPP) is a theory whic...
7,PCPIPCH,"Inflation rate, average consumer prices",World Economic Outlook (April 2025),Annual percent change,,The average consumer price index (CPI) is a me...
8,PCPIEPCH,"Inflation rate, end of period consumer prices",World Economic Outlook (April 2025),Annual percent change,,The end of period consumer price index (CPI) i...
9,LP,Population,World Economic Outlook (April 2025),Millions of people,,"Total population of a country, region, or grou..."


In [21]:
turkey_indicators

,indicator_id,indicator_label,source,unit,scale,description
0,NGDP_RPCH,Real GDP growth,World Economic Outlook (April 2025),Annual percent change,,Gross domestic product is the most commonly us...
1,NGDPD,"GDP, current prices",World Economic Outlook (April 2025),Billions of U.S. dollars,,Gross domestic product is the most commonly us...
2,NGDPDPC,"GDP per capita, current prices\n",World Economic Outlook (April 2025),U.S. dollars per capita,,Gross domestic product is the most commonly us...
3,PPPGDP,"GDP, current prices",World Economic Outlook (April 2025),Purchasing power parity; billions of internati...,,Gross domestic product is the most commonly us...
4,PPPPC,"GDP per capita, current prices",World Economic Outlook (April 2025),Purchasing power parity; international dollars...,,Gross domestic product (GDP) is the most commo...
5,PPPSH,"GDP based on PPP, share of world",World Economic Outlook (April 2025),Percent of World,,Purchasing Power Parity (PPP) weights are indi...
6,PPPEX,Implied PPP conversion rate,World Economic Outlook (April 2025),National currency per international dollar,,Purchasing Power Parity (PPP) is a theory whic...
7,PCPIPCH,"Inflation rate, average consumer prices",World Economic Outlook (April 2025),Annual percent change,,The average consumer price index (CPI) is a me...
8,PCPIEPCH,"Inflation rate, end of period consumer prices",World Economic Outlook (April 2025),Annual percent change,,The end of period consumer price index (CPI) i...
9,LP,Population,World Economic Outlook (April 2025),Millions of people,,"Total population of a country, region, or grou..."


In [22]:
import requests
import json
import pandas as pd
from tqdm import tqdm

def get_imf_data(url: str):
    """Generic function to fetch IMF API data"""
    response = requests.get(url)
    return json.loads(response.text)

def get_weo_indicators():
    """Get only World Economic Outlook indicators with metadata"""
    data = get_imf_data("https://www.imf.org/external/datamapper/api/v1/indicators")
    indicators = []
    for k, v in data["indicators"].items():
        # Only include indicators from WEO source
        if "World Economic Outlook" in v.get("source", ""):
            indicator = {
                "id": k,
                "label": v.get("label", ""),
                "source": v.get("source", ""),
                "unit": v.get("unit", ""),
                "scale": v.get("scale", ""),
                "description": v.get("description", "")
            }
            indicators.append(indicator)
    return pd.DataFrame(indicators)

def check_country_has_data(indicator_id: str, country_code: str = "TUR", year: str = "2023"):
    """Check if a specific indicator has data for the given country"""
    try:
        url = f"https://www.imf.org/external/datamapper/api/v1/{indicator_id}/{country_code}?periods={year}"
        data = get_imf_data(url)
        return bool(data.get("values", {}).get(indicator_id, {}).get(country_code, {}).get(year))
    except:
        return False

def get_turkey_weo_indicators(verbose: bool = True):
    """Get WEO indicators that contain data for Turkey"""
    weo_indicators = get_weo_indicators()
    turkey_weo_indicators = []
    
    iterable = tqdm(weo_indicators.itertuples(), total=len(weo_indicators)) if verbose else weo_indicators.itertuples()
    
    for indicator in iterable:
        if check_country_has_data(indicator.id):
            turkey_weo_indicators.append({
                "indicator_id": indicator.id,
                "indicator_label": getattr(indicator, "label", ""),
                "source": getattr(indicator, "source", ""),
                "unit": getattr(indicator, "unit", ""),
                "scale": getattr(indicator, "scale", ""), 
                "description": getattr(indicator, "description", "")
            })
    
    return pd.DataFrame(turkey_weo_indicators)


# Get only WEO indicators for Turkey
turkey_weo_indicators = get_turkey_weo_indicators()
print(f"Found {len(turkey_weo_indicators)} WEO indicators with data for Turkey")

# Save to Excel
turkey_weo_indicators.to_excel("turkey_weo_indicators.xlsx", index=False)

TypeError: argument of type 'NoneType' is not iterable

In [60]:
import requests
import json
import pandas as pd
from tqdm import tqdm

def get_imf_data(url: str):
    """Generic function to fetch IMF API data"""
    response = requests.get(url)
    return json.loads(response.text)

def get_weo_indicators():
    """Get only World Economic Outlook indicators with metadata"""
    data = get_imf_data("https://www.imf.org/external/datamapper/api/v1/indicators")
    indicators = []
    for k, v in data["indicators"].items():
        # Safely handle None source and check for WEO
        source = v.get("source") or ""  # Convert None to empty string
        if "World Economic Outlook" in source:
            indicator = {
                "id": k,
                "label": v.get("label", ""),
                "source": source,
                "unit": v.get("unit", ""),
                "scale": v.get("scale", ""),
                "description": v.get("description", "")
            }
            indicators.append(indicator)
    return pd.DataFrame(indicators)

def check_country_has_data(indicator_id: str, country_code: str = "TUR", year: str = "2023"):
    """Check if a specific indicator has data for the given country"""
    try:
        url = f"https://www.imf.org/external/datamapper/api/v1/{indicator_id}/{country_code}?periods={year}"
        data = get_imf_data(url)
        return bool(data.get("values", {}).get(indicator_id, {}).get(country_code, {}).get(year))
    except:
        return False

def get_turkey_weo_indicators(verbose: bool = True):
    """Get WEO indicators that contain data for Turkey"""
    weo_indicators = get_weo_indicators()
    turkey_weo_indicators = []
    
    iterable = tqdm(weo_indicators.itertuples(), total=len(weo_indicators)) if verbose else weo_indicators.itertuples()
    
    for indicator in iterable:
        if check_country_has_data(indicator.id):
            turkey_weo_indicators.append({
                "indicator_id": indicator.id,
                "indicator_label": getattr(indicator, "label", ""),
                "source": getattr(indicator, "source", ""),
                "unit": getattr(indicator, "unit", ""),
                "scale": getattr(indicator, "scale", ""), 
                "description": getattr(indicator, "description", "")
            })
    
    return pd.DataFrame(turkey_weo_indicators)

# Get only WEO indicators for Turkey
turkey_weo_indicators = get_turkey_weo_indicators()
print(f"Found {len(turkey_weo_indicators)} WEO indicators with data for Turkey")

# Save to Excel
turkey_weo_indicators.to_excel("turkey_weo_indicators.xlsx", index=False)

def get_imf_indicator_data(indicator_id: str, group_ids: list[str] = [], years: list[str] = []):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = "?periods=" + ",".join(years)

    response = requests.get(
        url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}"
    )
    response_body = json.loads(response.text)
    response_values = response_body.get("values")

    if not response_values:
        return pd.DataFrame()

    indicator_df = pd.DataFrame.from_records(
        response_body["values"][indicator_id]
    ).sort_index()
    
    return indicator_df


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:29<00:00,  1.95s/it]

Found 15 WEO indicators with data for Turkey


In [58]:
turkey_weo_indicators
countries = get_imf_groups('countries')

#indicator = indicators.loc[indicators.id == 'PCPIEPCH'].iloc[0]

selected_countries = countries.loc[countries.id.isin(['TUR'])]

for i in turkey_weo_indicators.indicator_id.unique():
    indicator_data = get_imf_indicator_data(indicator.id, selected_countries.id)

indicator_data.loc['2023':].T.to_excel("big_partners_.xl

SyntaxError: non-default argument follows default argument (1185162851.py, line 13)

In [55]:
indicator_data = get_imf_indicator_data("NGDPD", "TUR")

In [79]:
def get_imf_indicator_data(indicator_id: str, group_ids: list[str] = [], years: list[str] = []):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = "?periods=" + ",".join(years)

    response = requests.get(
        url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}"
    )
    response_body = json.loads(response.text)
    response_values = response_body.get("values")

    if not response_values:
        return pd.DataFrame()

    indicator_df = pd.DataFrame.from_records(
        response_body["values"][indicator_id]
    ).sort_index()
    
    return indicator_df

get_imf_indicator_data(indicator_id="NGDP_RPCH", group_ids=["Countries"], years= [2020]):

SyntaxError: invalid syntax (965134282.py, line 20)

In [70]:
def get_imf_indicators():
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    return indicators_df
get_imf_indicators()

,id,label,description,source,unit,dataset
0,NGDP_RPCH,Real GDP growth,Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Annual percent change,WEO
1,NGDPD,"GDP, current prices",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Billions of U.S. dollars,WEO
2,NGDPDPC,"GDP per capita, current prices\n",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),U.S. dollars per capita,WEO
3,PPPGDP,"GDP, current prices",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Purchasing power parity; billions of internati...,WEO
4,PPPPC,"GDP per capita, current prices",Gross domestic product (GDP) is the most commo...,World Economic Outlook (April 2025),Purchasing power parity; international dollars...,WEO
...,...,...,...,...,...,...
128,AI_PI,AI Preparedness Index,AI Preparedness Index,AI Preparedness Index - April 2024,Index,AIPI
129,DI,Digital Infrastructure,Digital Infrastructure,AI Preparedness Index - April 2024,Index,AIPI
130,IEI,Innovation and Economic Integration,Innovation and Economic Integration,AI Preparedness Index - April 2024,Index,AIPI
131,HCLMP,Human Capital and Labor Market Policies,Human Capital and Labor Market Policies,AI Preparedness Index - April 2024,Index,AIPI


In [74]:
def get_imf_groups(group_type):
    response = requests.get(
        url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}"
    )
    response_body = json.loads(response.text)
    results = [
        {"id": key, **values} for key, values in response_body[group_type].items()
    ]
    df = pd.DataFrame.from_records(results)
    return df



countries = get_imf_groups('countries')
countries

,id,label
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALB,Albania
...,...,...
236,AND,Andorra
237,IOT,British Indian Ocean Territories
238,BES,"Bonaire, Sint Eustatius and Saba"
239,KOS,Kosovo


In [82]:
import requests
import json
import pandas as pd

def get_imf_indicators(type_):
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    indicators_df=indicators_df[indicators_df["source"].str.contains(type_, na=False)]
    return indicators_df


def get_imf_groups(group_type):
    response = requests.get(
        url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}"
    )
    response_body = json.loads(response.text)
    results = [
        {"id": key, **values} for key, values in response_body[group_type].items()
    ]
    df = pd.DataFrame.from_records(results)
    return df

indicators = get_imf_indicators("World Economic Outlook")
indicators
countries = get_imf_groups('countries')
countries

,id,label
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALB,Albania
...,...,...
236,AND,Andorra
237,IOT,British Indian Ocean Territories
238,BES,"Bonaire, Sint Eustatius and Saba"
239,KOS,Kosovo


In [83]:
indicators

,id,label,description,source,unit,dataset
0,NGDP_RPCH,Real GDP growth,Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Annual percent change,WEO
1,NGDPD,"GDP, current prices",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Billions of U.S. dollars,WEO
2,NGDPDPC,"GDP per capita, current prices\n",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),U.S. dollars per capita,WEO
3,PPPGDP,"GDP, current prices",Gross domestic product is the most commonly us...,World Economic Outlook (April 2025),Purchasing power parity; billions of internati...,WEO
4,PPPPC,"GDP per capita, current prices",Gross domestic product (GDP) is the most commo...,World Economic Outlook (April 2025),Purchasing power parity; international dollars...,WEO
5,PPPSH,"GDP based on PPP, share of world",Purchasing Power Parity (PPP) weights are indi...,World Economic Outlook (April 2025),Percent of World,WEO
6,PPPEX,Implied PPP conversion rate,Purchasing Power Parity (PPP) is a theory whic...,World Economic Outlook (April 2025),National currency per international dollar,WEO
7,PCPIPCH,"Inflation rate, average consumer prices",The average consumer price index (CPI) is a me...,World Economic Outlook (April 2025),Annual percent change,WEO
8,PCPIEPCH,"Inflation rate, end of period consumer prices",The end of period consumer price index (CPI) i...,World Economic Outlook (April 2025),Annual percent change,WEO
9,LP,Population,"Total population of a country, region, or grou...",World Economic Outlook (April 2025),Millions of people,WEO


In [116]:
import requests
import json
import pandas as pd

def get_imf_indicators(type_):
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    indicators_df=indicators_df[indicators_df["source"].str.contains(type_, na=False)]
    return indicators_df


def get_imf_groups(group_type):
    response = requests.get(url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}")
    response_body = json.loads(response.text)
    results = [{"id": key, **values} for key, values in response_body[group_type].items()]
    df = pd.DataFrame.from_records(results)
    return df


def get_imf_indicator_data(indicator_id, group_ids, years=None):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = f"?periods={','.join(years)}" if years else ""
    response = requests.get(url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}")
    if response.status_code != 200:
        return pd.DataFrame()
    response_body = response.json()
    response_values = response_body.get("values")
    if not response_values or indicator_id not in response_values:
        return pd.DataFrame()
    all_country_data = response_values[indicator_id]
    indicator_df = pd.DataFrame.from_dict(all_country_data, orient="index").sort_index()
    indicator_df.index.name = "Country"
    return indicator_df

weo=get_imf_indicators("World Economic Outlook")


turkey_indicators_df = pd.DataFrame()

for i, label in zip(weo["id"], weo["label"]):
    try:
        weo_r = get_imf_indicator_data(i, group_ids=["WEO"])
        if "TUR" in weo_r.index:
            turkey_data = weo_r.loc["TUR"]
            turkey_data.name = label
            turkey_indicators_df[label] = turkey_data
        else:
            print(f"TUR not found in {i}")
    except Exception as e:
        print(f"Error processing {i} - {label}: {e}")

turkey_indicators_df.index.name = "Year"
turkey_indicators_df = turkey_indicators_df.sort_index()
turkey_indicators_df


,Real GDP growth,"GDP, current prices","GDP per capita, current prices\n","GDP per capita, current prices","GDP based on PPP, share of world",Implied PPP conversion rate,"Inflation rate, average consumer prices","Inflation rate, end of period consumer prices",Population,Current account balance\nU.S. dollars,"Current account balance, percent of GDP",Unemployment rate,General government net lending/borrowing,General government gross debt
Year,,,,,,,,,,,,,,
1980,-0.8,161.322,2148.973,3588.939,1.222,0.000,110.6,NaN,44.950,-3.090,-3.2,7.2,NaN,NaN
1981,4.4,184.293,2128.209,4007.704,1.246,0.000,36.4,NaN,45.985,-1.901,-1.9,7.2,NaN,NaN
1982,3.4,202.391,1890.083,4302.109,1.283,0.000,31.1,NaN,47.045,-0.952,-1.1,7.6,NaN,NaN
1983,4.8,220.323,1765.895,4579.003,1.310,0.000,31.3,NaN,48.116,-1.923,-2.3,7.5,NaN,NaN
1984,6.8,243.849,1680.418,4958.335,1.340,0.000,48.4,NaN,49.180,-1.439,-1.7,7.4,NaN,NaN
1985,4.3,262.273,1848.330,5222.266,1.348,0.000,44.5,NaN,50.222,-1.013,-1.1,6.9,NaN,NaN
1986,6.9,286.125,1997.258,5584.159,1.396,0.000,34.6,30.7,51.239,-1.465,-1.4,7.7,NaN,NaN
1987,10.0,322.616,2276.858,6176.532,1.479,0.000,38.9,55.0,52.233,-0.806,-0.7,8.1,NaN,NaN
1988,2.1,341.078,2349.943,6410.462,1.447,0.001,73.7,66.2,53.206,1.596,1.3,8.7,NaN,NaN


In [113]:
# Create empty DataFrame to store all indicators for Turkey
turkey_indicators_df = pd.DataFrame()

# Loop through each indicator
for i, label in zip(weo["id"], weo["label"]):
    try:
        weo_r = get_imf_indicator_data(i, group_ids=["WEO"])
        if "TUR" in weo_r.index:
            turkey_data = weo_r.loc["TUR"]
            turkey_data.name = label  # Name the series with the label
            turkey_indicators_df[label] = turkey_data
        else:
            print(f"TUR not found in {i}")
    except Exception as e:
        print(f"Error processing {i} - {label}: {e}")

# Transpose to have years as index if needed
turkey_indicators_df.index.name = "Year"
turkey_indicators_df = turkey_indicators_df.sort_index()

# Show the
